In [2]:
from datascience import *
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [3]:
cancer = Table.read_table('https://raw.githubusercontent.com/AUP-CS2091/class/master/week12/breast-cancer.csv')
credit = Table.read_table('https://raw.githubusercontent.com/AUP-CS2091/class/master/week13/credit.csv')

In [4]:
cancer_no_id = cancer.drop(0)
example = cancer_no_id.drop("Class").row(0)
training = cancer_no_id.exclude(0)
def distance(arr1, arr2):
    return np.sqrt(sum((arr1-arr2)**2))
def row_distance(row1,row2):
    return distance(np.array(row1),np.array(row2))
def compute_distances(training, example):
    distances = make_array()
    for row in training.drop("Class").rows:
        distances = np.append(distances, row_distance(row, example))
    return distances
def k_nearest_neighbours(training, example, k):
    distances = compute_distances(training, example)
    distance_table = training.with_column("Distance", distances)
    sorted_distance_table = distance_table.sort("Distance")
    return sorted_distance_table.take(np.arange(k))
def knn_classifier(training2, example2, k):
    neighbours = k_nearest_neighbours(training2, example2, k)
    return neighbours.group("Class").sort("count", descending=True).row(0).item(0)
knn_classifier(training, example,11)

0

In [6]:
example2 = cancer_no_id.drop("Class").row(550)
knn_classifier(training, example2,11)

1

In [7]:
cancer_training = cancer_no_id.take(np.arange(350))
cancer_test = cancer_no_id.take(np.arange(350, 683))

In [8]:
def estimate_accuracy(training, test, k):
    classifications_on_test = make_array()
    for row in test.drop("Class").rows:
        classification = knn_classifier(training, row, k)
        classifications_on_test = np.append(classifications_on_test, classification)
    return sum(1 - np.abs(classifications_on_test - test.column("Class"))) / test.num_rows
estimate_accuracy(cancer_training, cancer_test, 11)

0.990990990990991

In [23]:
estimate_accuracy(cancer_training, cancer_test, 1)

0.9819819819819819

In [10]:
estimate_accuracy(cancer_training, cancer_training, 1)

1.0

In [11]:
estimate_accuracy(cancer_training, cancer_training, 11)

0.9628571428571429

In [20]:
credit = credit.drop("LIMIT_BAL")

In [34]:
credit_shuffed = credit.sample(with_replacement=False)
credit_training = credit_shuffed.take(np.arange(500))
credit_test = credit_shuffed.take(np.arange(500,1000))

In [35]:
estimate_accuracy(credit_training, credit_test, 1)

0.524

In [37]:
estimate_accuracy(credit_training, credit_training, 3)

0.594

In [18]:
row = credit.drop("Class").row(600)
knn_classifier(credit_training, row, 1)

0

In [ ]:
row

In [19]:
credit.row(600)

Row(LIMIT_BAL=100000, PAY_0=3, PAY_2=2, PAY_3=2, PAY_4=2, PAY_5=0, PAY_6=0, Class=1)

In [24]:
credit.show(5)

PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Class
1,2,2,2,2,2,0
0,0,0,0,0,0,0
5,4,3,2,2,2,0
0,0,0,0,0,2,0
0,0,0,0,0,0,0


In [29]:
credit_training.group("Class")

Class,count
0,500


In [30]:
credit_test.group("Class")

Class,count
1,500


In [38]:
# Bayes

In [38]:
n = 100
second = round(n * 0.6)
third = round(n * 0.4)

year = np.array(['Second'] * second + ['Third'] * third)
major = np.array(['Declared'] * (round(second * 0.5)) + ['Undeclared'] * (round(second * 0.5)) + \
                 ['Declared'] * (round(third * 0.8))  + ['Undeclared'] * (round(third * 0.2)))
                 
students = Table().with_columns(
    'Year', year,
    'Major', major
)
students.show(3)

Year,Major
Second,Declared
Second,Declared
Second,Declared


In [41]:
students.group(["Year", "Major"])

Year,Major,count
Second,Declared,30
Second,Undeclared,30
Third,Declared,32
Third,Undeclared,8


In [42]:
students.pivot("Year","Major")

Major,Second,Third
Declared,30,32
Undeclared,30,8
